# ODE solver examples

demonstration of various ODE solution methods including Van der Pol oscillator

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
from numeric_integrator import solve_ode

print("library loaded successfully")

## example 1: exponential growth

solve $\frac{dy}{dx} = y$, $y(0) = 1$, exact solution: $y = e^x$

In [ ]:
# define ODE
def f(x, y):
    return y

y0 = 1.0
x0 = 0.0
x_end = 2.0

# solve with different methods
methods = ["euler", "heun", "rk4", "rkf45"]
solutions = {}

for method in methods:
    if method == "rkf45":
        sol = solve_ode(f, y0, x0, x_end, method=method, tol=1e-6)
    else:
        sol = solve_ode(f, y0, x0, x_end, method=method, step=0.1)
    solutions[method] = sol
    error = abs(sol.y[-1] - np.exp(x_end))
    print(f"{method:8s}: y({x_end})={sol.y[-1]:.6f}, error={error:.2e}, evals={sol.n_evaluations}")

print(f"\nexact:    y({x_end})={np.exp(x_end):.6f}")

## example 2: method comparison plot

In [ ]:
# plot solutions
plt.figure(figsize=(12, 5))

# left: solutions
plt.subplot(1, 2, 1)
x_exact = np.linspace(x0, x_end, 200)
y_exact = np.exp(x_exact)
plt.plot(x_exact, y_exact, 'k-', linewidth=2, label='exact', alpha=0.5)

for method, sol in solutions.items():
    plt.plot(sol.x, sol.y, 'o-', label=method, markersize=4)

plt.xlabel('x', fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title('ODE solutions', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

# right: errors
plt.subplot(1, 2, 2)
for method, sol in solutions.items():
    errors = np.abs(sol.y - np.exp(sol.x))
    plt.semilogy(sol.x, errors, 'o-', label=method, markersize=4)

plt.xlabel('x', fontsize=12)
plt.ylabel('absolute error', fontsize=12)
plt.title('error comparison', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## example 3: Van der Pol oscillator

solve the Van der Pol equation:

$$\frac{d^2y}{dt^2} - \mu(1-y^2)\frac{dy}{dt} + y = 0$$

convert to first-order system:
- $y_1 = y$
- $y_2 = \frac{dy}{dt}$
- $\frac{dy_1}{dt} = y_2$
- $\frac{dy_2}{dt} = \mu(1-y_1^2)y_2 - y_1$

In [ ]:
def van_der_pol(t, y, mu=1.0):
    """Van der Pol oscillator as first-order system"""
    y1, y2 = y
    dy1dt = y2
    dy2dt = mu * (1 - y1**2) * y2 - y1
    return np.array([dy1dt, dy2dt])

# initial conditions
y0 = np.array([2.0, 0.0])  # [position, velocity]
t0 = 0.0
t_end = 30.0

# solve for different mu values
mu_values = [0.5, 1.0, 2.0]
solutions_vdp = {}

for mu in mu_values:
    f_mu = lambda t, y: van_der_pol(t, y, mu=mu)
    sol = solve_ode(f_mu, y0, t0, t_end, method="rk4", step=0.05)
    solutions_vdp[mu] = sol
    print(f"μ={mu}: solved with {len(sol.x)} points")

## example 4: visualize Van der Pol oscillator

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# time series
ax = axes[0, 0]
for mu, sol in solutions_vdp.items():
    y1 = sol.y[:, 0] if sol.y.ndim > 1 else sol.y
    ax.plot(sol.x, y1, label=f'μ={mu}', linewidth=2)
ax.set_xlabel('time (t)', fontsize=12)
ax.set_ylabel('position (y)', fontsize=12)
ax.set_title('time series', fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# phase portraits
ax = axes[0, 1]
for mu, sol in solutions_vdp.items():
    if sol.y.ndim > 1:
        y1, y2 = sol.y[:, 0], sol.y[:, 1]
    else:
        continue
    ax.plot(y1, y2, label=f'μ={mu}', linewidth=2)
ax.set_xlabel('position (y)', fontsize=12)
ax.set_ylabel('velocity (dy/dt)', fontsize=12)
ax.set_title('phase portrait', fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# individual phase portraits
for idx, (mu, sol) in enumerate(solutions_vdp.items()):
    if idx >= 2:
        break
    ax = axes[1, idx]
    if sol.y.ndim > 1:
        y1, y2 = sol.y[:, 0], sol.y[:, 1]
        ax.plot(y1, y2, linewidth=2, color=f'C{idx}')
        ax.plot(y1[0], y2[0], 'go', markersize=10, label='start')
        ax.plot(y1[-1], y2[-1], 'ro', markersize=10, label='end')
    ax.set_xlabel('position (y)', fontsize=12)
    ax.set_ylabel('velocity (dy/dt)', fontsize=12)
    ax.set_title(f'μ={mu}', fontsize=13)
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nVan der Pol oscillator exhibits limit cycle behavior")
print("larger μ values produce more nonlinear oscillations")

## example 5: adaptive vs fixed step

In [ ]:
# stiff-like behavior with mu=5
f_stiff = lambda t, y: van_der_pol(t, y, mu=5.0)

# fixed step
sol_fixed = solve_ode(f_stiff, y0, 0, 20, method="rk4", step=0.1)

# adaptive step
sol_adaptive = solve_ode(f_stiff, y0, 0, 20, method="rkf45", tol=1e-4)

print(f"fixed step (h=0.1):   {len(sol_fixed.x)} points")
print(f"adaptive (tol=1e-4):  {len(sol_adaptive.x)} points")
print(f"adaptive method uses fewer points by adapting to solution behavior")

# plot
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
if sol_fixed.y.ndim > 1:
    plt.plot(sol_fixed.x, sol_fixed.y[:, 0], 'o-', label='fixed', markersize=3)
if sol_adaptive.y.ndim > 1:
    plt.plot(sol_adaptive.x, sol_adaptive.y[:, 0], 's-', label='adaptive', markersize=3)
plt.xlabel('time')
plt.ylabel('position')
plt.title('Van der Pol (μ=5)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
step_sizes = np.diff(sol_adaptive.x)
plt.plot(sol_adaptive.x[1:], step_sizes, 'o-', markersize=4)
plt.xlabel('time')
plt.ylabel('step size')
plt.title('adaptive step size variation')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## example 6: multi-step methods

In [ ]:
# simple ODE for testing multi-step
f_simple = lambda x, y: -2 * x * y
y0_simple = 1.0

# compare Adams methods
sol_ab2 = solve_ode(f_simple, y0_simple, 0, 2, method="ab2", step=0.1)
sol_ab3 = solve_ode(f_simple, y0_simple, 0, 2, method="ab3", step=0.1)
sol_am2 = solve_ode(f_simple, y0_simple, 0, 2, method="am2", step=0.1)

# exact solution: y = exp(-x^2)
x_exact = np.linspace(0, 2, 200)
y_exact = np.exp(-x_exact**2)

plt.figure(figsize=(10, 6))
plt.plot(x_exact, y_exact, 'k-', linewidth=2, label='exact', alpha=0.5)
plt.plot(sol_ab2.x, sol_ab2.y, 'o-', label='Adams-Bashforth-2', markersize=5)
plt.plot(sol_ab3.x, sol_ab3.y, 's-', label='Adams-Bashforth-3', markersize=5)
plt.plot(sol_am2.x, sol_am2.y, '^-', label='Adams-Moulton-2', markersize=5)
plt.xlabel('x', fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title('multi-step methods comparison', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

print("Adams-Moulton (implicit) typically more accurate than Adams-Bashforth (explicit)")